# Explicación de variables por gesto

In [19]:
# importar librerias
import pandas as pd
import joblib

import warnings
from sklearn.exceptions import InconsistentVersionWarning

# Suppress the specific warning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)


In [20]:
#---------Importar los datos-----------
# Dataframe medidas calculadas por repetición
df = pd.read_csv('../Resultados/medidasPerRepetition.csv', dtype=object)


In [21]:
gestures = df['GestureName'].unique()

with open('../Resultados/FS_x_gesture.txt', 'w') as file:
    for gesture in gestures:
        modelo_gesto_path = f'../Resultados/modelo_{gesture}.sav'
        best_pipeline, expected_columns = joblib.load(modelo_gesto_path)

        # Step 1: Get feature names from the 'preprocessing' step
        preprocessor = best_pipeline.named_steps['preprocessing']

        # Extract the feature names after preprocessing (OrdinalEncoder + remainder columns)
        feature_names = preprocessor.get_feature_names_out()

        # Step 2: Access the SelectKBest step and get the selected features
        select_kbest = best_pipeline.named_steps['select_features']

        # Get the boolean mask for the selected features
        selected_features_mask = select_kbest.get_support()

        # Use the mask to extract the selected feature names
        selected_features = feature_names[selected_features_mask]
        selected_features = [name.replace('remainder__', '') for name in selected_features]


        file.write(f'\n------------ VARIABLES GESTO {gesture} -----------\n')

        for row in selected_features:
            file.write(''.join(map(str, row)) + '\n')


